<a href="https://colab.research.google.com/github/Aasthapaudel/llma-blog-gen-try/blob/main/End_To_End_Blog_Generation_LLM_Platform_End_To_End_LLM_Project_Using_LLAMA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
%%shell

#!/usr/bin/env bash

# Copyright (c) Meta Platforms, Inc. and affiliates.
# This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

set -e

PRESIGNED_URL="https://download.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoienZneG16anF1Z2Rxa2c3NDAycmJqZ2U4IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZG93bmxvYWQubGxhbWFtZXRhLm5ldFwvKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjcxNzAwNX19fV19&Signature=KWJ0Qf4SqLtGxdy87lCYip1x0nUi72szpntn6WutneH9rIm9VHIMjEe6LkNhAHrXj1lgHE2Cxsm66Ezf%7E2dEwWTBrbKSti3GPXj8DnhqlWNeiM%7EG6iayR7MIM2qcsVngmtgOtTXY68s03BGov6GUBTME4JCN2bD%7EjO0ASBnBG3q1APAG9yhkqNJ8f0cs%7EmwQ8A-FrlZulgEa3lFuW3ZiVetoD0hiKaWEwn-9b4hXeDZ5VeOs%7EhlGCSVg%7EQBEhRTxlMCuX1SavDh4lbvxIb%7EYfy9vWzslpxaQsdWvcCBvs4QRSoiGprcv9RUuDAZ7bcuYP%7EvLznOPe%7E1DEAsAgNknBQ__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1478935189379764"

TARGET_FOLDER="."             # where all files should end up
mkdir -p "${TARGET_FOLDER}"

echo "Downloading LICENSE and Acceptable Usage Policy"
wget --continue "${PRESIGNED_URL/'*'/'LICENSE'}" -O "${TARGET_FOLDER}/LICENSE"
wget --continue "${PRESIGNED_URL/'*'/'USE_POLICY.md'}" -O "${TARGET_FOLDER}/USE_POLICY.md"

echo "Downloading tokenizer"
wget --continue "${PRESIGNED_URL/'*'/'tokenizer.model'}" -O "${TARGET_FOLDER}/tokenizer.model"
wget --continue "${PRESIGNED_URL/'*'/'tokenizer_checklist.chk'}" -O "${TARGET_FOLDER}/tokenizer_checklist.chk"
CPU_ARCH=$(uname -m)
if [ "${CPU_ARCH}" = "arm64" ]; then
    (cd "${TARGET_FOLDER}" && md5sum tokenizer_checklist.chk)
else
    (cd "${TARGET_FOLDER}" && md5sum -c tokenizer_checklist.chk)
fi

for m in "7B" "7B-chat"
do
    if [[ "${m}" == "7B" ]]; then
        SHARD=0
        MODEL_PATH="llama-2-7b"
    elif [[ "${m}" == "7B-chat" ]]; then
        SHARD=0
        MODEL_PATH="llama-2-7b-chat"
    fi

    echo "Downloading ${MODEL_PATH}"
    mkdir -p "${TARGET_FOLDER}/${MODEL_PATH}"

    for s in $(seq -f "0%g" 0 "${SHARD}")
    do
        wget --continue "${PRESIGNED_URL/'*'/"${MODEL_PATH}/consolidated.${s}.pth"}" -O "${TARGET_FOLDER}/${MODEL_PATH}/consolidated.${s}.pth"
    done

    wget --continue "${PRESIGNED_URL/'*'/"${MODEL_PATH}/params.json"}" -O "${TARGET_FOLDER}/${MODEL_PATH}/params.json"
    wget --continue "${PRESIGNED_URL/'*'/"${MODEL_PATH}/checklist.chk"}" -O "${TARGET_FOLDER}/${MODEL_PATH}/checklist.chk"
    echo "Checking checksums"
    if [ "${CPU_ARCH}" = "arm64" ]; then
      (cd "${TARGET_FOLDER}/${MODEL_PATH}" && md5sum checklist.chk)
    else
      (cd "${TARGET_FOLDER}/${MODEL_PATH}" && md5sum -c checklist.chk)
    fi
done


--2024-05-25 11:46:25--  https://download.llamameta.net/LICENSE?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoienZneG16anF1Z2Rxa2c3NDAycmJqZ2U4IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZG93bmxvYWQubGxhbWFtZXRhLm5ldFwvKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjcxNzAwNX19fV19&Signature=KWJ0Qf4SqLtGxdy87lCYip1x0nUi72szpntn6WutneH9rIm9VHIMjEe6LkNhAHrXj1lgHE2Cxsm66Ezf%7E2dEwWTBrbKSti3GPXj8DnhqlWNeiM%7EG6iayR7MIM2qcsVngmtgOtTXY68s03BGov6GUBTME4JCN2bD%7EjO0ASBnBG3q1APAG9yhkqNJ8f0cs%7EmwQ8A-FrlZulgEa3lFuW3ZiVetoD0hiKaWEwn-9b4hXeDZ5VeOs%7EhlGCSVg%7EQBEhRTxlMCuX1SavDh4lbvxIb%7EYfy9vWzslpxaQsdWvcCBvs4QRSoiGprcv9RUuDAZ7bcuYP%7EvLznOPe%7E1DEAsAgNknBQ__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1478935189379764
Resolving download.llamameta.net (download.llamameta.net)... 18.238.217.58, 18.238.217.71, 18.238.217.67, ...
Connecting to download.llamameta.net (download.llamameta.net)|18.238.217.58|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Sa

In [ ]:
!pip install sentence-transformers
!pip install uvicorn
!pip install ctransformers
!pip install langchain
!pip install python-box
!pip install streamlit


In [ ]:
!pip install streamlit
!pip install pyngrok


In [ ]:
!pip install langchain


In [ ]:
pip install langchain_community

In [ ]:
from langchain.llms import CTransformers


In [ ]:
import streamlit as st
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

## Function To get response from LLAma 2 model

def getLLamaresponse(input_text,no_words,blog_style):

    ### LLama2 model
    llm=CTransformers(model='models/llama-2-7b-chat.ggmlv3.q8_0.bin',
                      model_type='llama',
                      config={'max_new_tokens':256,
                              'temperature':0.01})

    ## Prompt Template

    template="""
        Write a blog for {blog_style} job profile for a topic {input_text}
        within {no_words} words.
            """

    prompt=PromptTemplate(input_variables=["blog_style","input_text",'no_words'],
                          template=template)

    ## Generate the response from the LLama 2 model
    response=llm(prompt.format(blog_style=blog_style,input_text=input_text,no_words=no_words))
    print(response)
    return response

st.set_page_config(page_title="Generate Blogs",
                    page_icon='🤖',
                    layout='centered',
                    initial_sidebar_state='collapsed')

st.header("Generate Blogs 🤖")

input_text=st.text_input("Enter the Blog Topic")

## creating two more columns for additional 2 fields

col1,col2=st.columns([5,5])

with col1:
    no_words=st.text_input('No of Words')
with col2:
    blog_style=st.selectbox('Writing the blog for',
                            ('Researchers','Data Scientist','Common People'),index=0)

submit=st.button("Generate")

## Final response
if submit:
    st.write(getLLamaresponse(input_text,no_words,blog_style))


In [22]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.123.184:8501

  Stopping...
  Stopping...
